# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 本問は **仕様で降順指定あり**（`ORDER BY rating DESC`）
* `NOT IN` は NULL 罠のため回避（本問では不要）
* 判定は **ID 基準**、表示は仕様どおりの列名と順序

## 1) 問題

* `映画テーブル Cinema から、ID が奇数かつ description が "boring" ではない映画を抽出し、rating の降順で返せ。`
* 入力テーブル例: `Cinema(id, movie, description, rating)`
* 出力仕様: `id, movie, description, rating` を **rating 降順**で返す

## 2) 最適解（単一クエリ）

> 本問はウィンドウ不要。単純な条件抽出＋降順ソートで 1 クエリ。

```sql
SELECT
  id,
  movie,
  description,
  rating
FROM Cinema
WHERE (id % 2) = 1          -- 奇数IDのみ
  AND description <> 'boring' -- 文字列一致で除外（NULL は自動的に除外される）
ORDER BY rating DESC;         -- 仕様で降順指定

Runtime 241 ms
Beats 34.15%

```

## 3) 代替解

> 同義。奇数判定をビット演算にするだけ。実行計画・結果は同等。

```sql
SELECT
  id,
  movie,
  description,
  rating
FROM Cinema
WHERE (id & 1) = 1
  AND description <> 'boring'
ORDER BY rating DESC;

Runtime 249 ms
Beats 25.20%

```

## 4) 要点解説

* **奇数判定**: `id % 2 = 1` でも `(id & 1) = 1` でも可。整数主キーならどちらも安全。
* **"boring" 除外**: `<> 'boring'` は `description IS NULL` 行を含めない（`UNKNOWN` のため WHERE で落ちる）。NULL を含めたいなら `OR description IS NULL` を追加する。
* **順序**: 本テンプレでは「順不同を優先」とあるが、本問は仕様で **降順必須** のため `ORDER BY rating DESC` を入れる。

## 5) 計算量（概算）

* テーブルフルスキャン時: **O(N)**。`id`（PK）の演算は行ごと定数時間、`description <> 'boring'` も行ごと定数時間。
* インデックス: 本条件は選択性が低いので索引効果は限定的（`rating` で並び替えるため filesort が走りやすい）。

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Cinema] --> B[条件抽出 id 奇数]
  B --> C["条件抽出 description <> \"boring\""]
  C --> D[rating 降順で並べ替え]
  D --> E[出力 id, movie, description, rating]
```

この問題の範囲（クエリだけで勝負、スキーマ変更なし）だと、ほぼ最短距離が書けています。差分 241ms vs 249ms は誤差レベルで、`%2` と `&1` の優劣は測定ノイズの域です。
それでも「もう少しだけ良くする／明確にする」観点で、実用的な改善ポイントを挙げます。

## クエリだけでできる微調整

1. **順序の決定性を高める（同率タイを安定化）**
   rating が同じ映画がある場合、返却順が揺れます。審査系や再現性重視ならタイブレークを追加。

```sql
SELECT id, movie, description, rating
FROM Cinema
WHERE (id & 1) = 1
  AND description <> 'boring'
ORDER BY rating DESC, id DESC;  -- 安定化

Runtime 222 ms
Beats 67.10%

```

※速度影響はほぼ無し、可読性と再現性の向上。

2. **NULL の扱いを明示**
   仕様どおり「boring ではない」を厳密に“NULL は含めない”と解釈するなら明示しておくと安心。

```sql
... AND description IS NOT NULL
    AND description <> 'boring'
```

逆に **NULL も許可**したいなら `OR description IS NULL` を併記。

3. **大文字小文字の扱いを固定**
   環境の照合順序で “Boring” などの扱いが変わります。区別したいならバイナリ比較に。

```sql
... AND BINARY description <> 'boring'
-- もしくは
... AND description COLLATE utf8mb4_bin <> 'boring'
```

> ここまでが LeetCode 的に現実的な「クエリ単体」改善。速度はほぼ変わらず、**結果の安定性・意図の明確さ**が主効果です。

## スキーマ変更が許される現場向け最適化（参考）

> テーブルが大きく、実運用で速度を突き詰めたい場合。

* **filesort 回避用インデックス**（`ORDER BY rating DESC` をインデックス順で満たす）

  ```sql
  CREATE INDEX idx_cinema_rating_desc ON Cinema (rating DESC, id);
  ```

  *効果*: 並べ替えコストを大幅削減（ただし `description <> 'boring'` は残差条件で評価）。
  *ポイント*: 取り出し列 `movie, description` は二次索引から PK 経由でルックアップされます（InnoDB 仕様）。

* **関数インデックスで偶数/奇数を sargable に**

  ```sql
  CREATE INDEX idx_cinema_odd_rating ON Cinema ( (id & 1), rating DESC );
  -- そしてクエリは WHERE (id & 1) = 1 AND description <> 'boring'
  ```

  *効果*: まず `(id & 1)=1` で範囲を半減 → そのまま `rating DESC` でインデックススキャンし、
  `description <> 'boring'` はフィルタで落とす。大規模データで効きやすいです。

* **description 側の選択性が高い場合**
  “boring” が多い/少ないで有効度が変わりますが、**読み取り量を減らす**方向に張るなら：

  ```sql
  CREATE INDEX idx_cinema_desc_rating ON Cinema (description, rating DESC);
  ```

  *注意*: `<> 'boring'` はレンジ分割（`< 'boring'` と `> 'boring'`）になるため、
  インデックス効用はデータ分布に強く依存します。実測＆EXPLAIN で判断を。

## まとめ

* **クエリ単体では既に最適レベル**。実測 241ms と 249ms の差は誤差で、どちらも OK。
* 品質面の小改善は **タイブレーク追加**・**NULL/大文字小文字の扱いを明示**。
* 実運用で速度を上げるなら **`ORDER BY` 用の降順インデックス** と **(id & 1) の関数インデックス**が効きます。

必要なら、あなたの想定データ量・分布を仮定して `EXPLAIN` の読み方とインデックス案をもう少し踏み込みで出します。

